## Data Science (Assignment 1)
### YELP Dataset

##### 1) Installing required libraries

In [1]:
!pip install pandas json5

Defaulting to user installation because normal site-packages is not writeable


##### 2) Loading required libraries

In [2]:
import json
import sqlite3
import pandas as pd
import os

##### 3) Loading all the files of YELP Dataset

In [3]:
json_files = {
    "business": "data/business.json",
    "review": "data/review.json",
    "user": "data/user.json",
    "checkin": "data/checkin.json",
    "tip": "data/tip.json"
}

def load_json(file_path):
    """Loads JSON data from a given file path."""
    with open(file_path, "r", encoding="utf-8") as file:
        return [json.loads(line) for line in file]

# Load data
yelp_data = {key: load_json(path) for key, path in json_files.items()}
print("Loaded Yelp JSON data successfully!")

Loaded Yelp JSON data successfully!


##### 4) Pre-Processing and Cleaning the Datasets

In [ ]:
def extract_business_data(business_json):
    business_data = []
    for biz in business_json:
        if not biz.get("business_id") or not biz.get("name"):
            continue  # Skip records missing essential fields

        business_data.append({
            "business_id": biz.get("business_id"),
            "name": biz.get("name"),
            "city": biz.get("city", "Unknown"),  
            "rating": biz.get("stars", 0),
            "review_count": biz.get("review_count", 0),
            "latitude": biz.get("latitude", None),
            "longitude": biz.get("longitude", None),
            "categories": biz.get("categories", "").split(", ") if biz.get("categories") else []
        })
    return business_data

def extract_review_data(review_json):
    return [{
        "review_id": rev.get("review_id"),
        "user_id": rev.get("user_id"),
        "business_id": rev.get("business_id"),
        "stars": rev.get("stars"),
        "date": rev.get("date")
    } for rev in review_json if rev.get("review_id") and rev.get("business_id")]

def extract_user_data(user_json):
    return [{
        "user_id": user.get("user_id"),
        "name": user.get("name", "Unknown"),
        "review_count": user.get("review_count", 0)
    } for user in user_json if user.get("user_id")]

def extract_checkin_data(checkin_json):
    return [{
        "business_id": chk.get("business_id"),
        "date": chk.get("date")
    } for chk in checkin_json if chk.get("business_id")]

def extract_tip_data(tip_json):
    return [{
        "user_id": tip.get("user_id"),
        "business_id": tip.get("business_id"),
        "text": tip.get("text"),
        "date": tip.get("date")
    } for tip in tip_json if tip.get("user_id") and tip.get("business_id")]

# Convert processed data into DataFrames
df_business = pd.DataFrame(extract_business_data(yelp_data["business"]))
df_review = pd.DataFrame(extract_review_data(yelp_data["review"]))
df_user = pd.DataFrame(extract_user_data(yelp_data["user"]))
df_checkin = pd.DataFrame(extract_checkin_data(yelp_data["checkin"]))
df_tip = pd.DataFrame(extract_tip_data(yelp_data["tip"]))

df_business.head()

,business_id,name,city,rating,review_count,latitude,longitude,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",Santa Barbara,5.0,7,34.426679,-119.711197,"[Doctors, Traditional Chinese Medicine, Naturo..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,Affton,3.0,15,38.551126,-90.335695,"[Shipping Centers, Local Services, Notaries, M..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,Tucson,3.5,22,32.223236,-110.880452,"[Department Stores, Shopping, Fashion, Home & ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,4.0,80,39.955505,-75.155564,"[Restaurants, Food, Bubble Tea, Coffee & Tea, ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,Green Lane,4.5,13,40.338183,-75.471659,"[Brewpubs, Breweries, Food]"


##### 5) Creating SQL Databases and Tables

In [ ]:
DB_PATH = "data/YelpDB.db"
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

schema_sql = '''
CREATE TABLE IF NOT EXISTS Business (
    business_id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    city TEXT NOT NULL,  
    rating REAL CHECK (rating BETWEEN 0 AND 5),
    review_count INTEGER CHECK (review_count >= 0),
    latitude REAL,
    longitude REAL
);

CREATE TABLE IF NOT EXISTS Category (
    category_id INTEGER PRIMARY KEY AUTOINCREMENT,
    business_id TEXT,
    category_name TEXT NOT NULL,
    FOREIGN KEY (business_id) REFERENCES Business(business_id) ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Review (
    review_id TEXT PRIMARY KEY,
    user_id TEXT,
    business_id TEXT,
    stars INTEGER CHECK (stars BETWEEN 1 AND 5),
    date TEXT,
    FOREIGN KEY (business_id) REFERENCES Business(business_id),
    FOREIGN KEY (user_id) REFERENCES User(user_id)
);

CREATE TABLE IF NOT EXISTS User (
    user_id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    review_count INTEGER CHECK (review_count >= 0)
);

CREATE TABLE IF NOT EXISTS Checkin (
    checkin_id INTEGER PRIMARY KEY AUTOINCREMENT,
    business_id TEXT,
    date TEXT,
    FOREIGN KEY (business_id) REFERENCES Business(business_id)
);

CREATE TABLE IF NOT EXISTS Tip (
    tip_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id TEXT,
    business_id TEXT,
    text TEXT,
    date TEXT,
    FOREIGN KEY (business_id) REFERENCES Business(business_id),
    FOREIGN KEY (user_id) REFERENCES User(user_id)
);
'''

# Define schema file path
SCHEMA_PATH = "sql/schema.sql"

# Ensure sql folder exists (but don't recreate if it already does)
os.makedirs("sql", exist_ok=True)

# Only save schema if schema.sql doesn't already exist
if not os.path.exists(SCHEMA_PATH):
    with open(SCHEMA_PATH, "w", encoding="utf-8") as f:
        f.write(schema_sql)
    print(f"Database schema saved to {SCHEMA_PATH}")
else:
    print("schema.sql already exists, skipping file creation.")


# Execute schema in SQLite
cursor.executescript(schema_sql)
conn.commit()
print("Database and tables created successfully!")

Database schema saved to sql/schema.sql
Database and tables created successfully!


##### 6) Inserting Cleaned Data into Created Databases

In [ ]:
# Ensure connection to the database
conn = sqlite3.connect("data/YelpDB.db")
cursor = conn.cursor()

# Insert Business & Category Data
for _, row in df_business.iterrows():
    cursor.execute("""
        INSERT OR IGNORE INTO Business (business_id, name, city, rating, review_count, latitude, longitude)
        VALUES (?, ?, ?, ?, ?, ?, ?)""",
        (row["business_id"], row["name"], row["city"], row["rating"], row["review_count"], row["latitude"], row["longitude"]))

    # Insert Categories (if any)
    for category in row["categories"]:
        cursor.execute("""
            INSERT OR IGNORE INTO Category (business_id, category_name)
            VALUES (?, ?)""",
            (row["business_id"], category))

# Insert User Data
for _, row in df_user.iterrows():
    cursor.execute("""
        INSERT OR IGNORE INTO User (user_id, name, review_count)
        VALUES (?, ?, ?)""",
        (row["user_id"], row["name"], row["review_count"]))

# Insert Review Data
for _, row in df_review.iterrows():
    cursor.execute("""
        INSERT OR IGNORE INTO Review (review_id, user_id, business_id, stars, date)
        VALUES (?, ?, ?, ?, ?)""",
        (row["review_id"], row["user_id"], row["business_id"], row["stars"], row["date"]))

# Insert Checkin Data
for _, row in df_checkin.iterrows():
    cursor.execute("""
        INSERT OR IGNORE INTO Checkin (business_id, date)
        VALUES (?, ?)""",
        (row["business_id"], row["date"]))

# Insert Tip Data
for _, row in df_tip.iterrows():
    cursor.execute("""
        INSERT OR IGNORE INTO Tip (user_id, business_id, text, date)
        VALUES (?, ?, ?, ?)""",
        (row["user_id"], row["business_id"], row["text"], row["date"]))

# Commit the changes
conn.commit()
print("All data inserted successfully!")

All data inserted successfully!


##### 7) Running Queries to Validate

In [ ]:
# Printing the total number of records in each table
tables = ["Business", "Category", "User", "Review", "Checkin", "Tip"]

for table in tables:
    query = f"SELECT COUNT(*) as total_records FROM {table};"
    df_count = pd.read_sql(query, conn)
    print(f"Total records in {table}: {df_count.iloc[0,0]}")


Total records in Business: 150346
Total records in Category: 668592
Total records in User: 1987897
Total records in Review: 6990280
Total records in Checkin: 131930
Total records in Tip: 908915


In [ ]:
# Restaurants with reviews count more than 50
conn = sqlite3.connect("data/YelpDB.db")
query = """
    SELECT name, city, rating, review_count 
    FROM Business 
    WHERE review_count > 50  
    ORDER BY rating DESC, review_count DESC
    LIMIT 10;
"""
df_top_rated_businesses = pd.read_sql(query, conn)
df_top_rated_businesses

,name,city,rating,review_count
0,Blues City Deli,Saint Louis,5.0,991
1,Carlillos Cocina,Sparks,5.0,799
2,Free Tours By Foot,New Orleans,5.0,769
3,Tumerico,Tucson,5.0,705
4,Yats,Franklin,5.0,623
5,Nelson's Green Brier Distillery,Nashville,5.0,545
6,Smiling With Hope Pizza,Reno,5.0,526
7,Barracuda Deli Cafe St. Pete Beach,St Pete Beach,5.0,521
8,SUGARED + BRONZED,Philadelphia,5.0,513
9,Cafe Soleil,St. Pete Beach,5.0,468


In [ ]:
# Restaurants in a city with highest reviews in descending order
conn = sqlite3.connect("data/YelpDB.db")
city_name = "Tucson"  
query = """
    SELECT name, city, rating, review_count 
    FROM Business 
    WHERE city = ? 
    ORDER BY review_count DESC 
    LIMIT 10;
"""
df_most_reviewed = pd.read_sql(query, conn, params=(city_name,))
df_most_reviewed

,name,city,rating,review_count
0,Prep & Pastry,Tucson,4.5,2126
1,El Charro Cafe,Tucson,3.5,1583
2,Cafe Poca Cosa,Tucson,4.0,1306
3,Culinary Dropout,Tucson,4.0,1295
4,HUB Restaurant & Ice Creamery,Tucson,4.0,1270
5,The Parish,Tucson,4.5,1210
6,Guadalajara Original Grill,Tucson,4.0,1117
7,Baja Cafe,Tucson,4.5,1074
8,Serial Grillers,Tucson,4.5,986
9,Wild Garlic Grill,Tucson,3.5,968


In [19]:
# Cities with highest average business ratings
conn = sqlite3.connect("data/YelpDB.db")
query = """
    SELECT city, 
           ROUND(AVG(rating), 2) AS avg_rating, 
           COUNT(*) AS total_businesses 
    FROM Business 
    GROUP BY city 
    HAVING COUNT(*) > 20  -- Ensures only cities with enough businesses are included
    ORDER BY avg_rating DESC
    LIMIT 10;
"""
df_best_cities = pd.read_sql(query, conn)
df_best_cities

,city,avg_rating,total_businesses
0,Virginia City,4.33,32
1,Montecito,4.16,93
2,Washington Crossing,4.14,22
3,Wyndmoor,4.13,27
4,Safety Harbor,4.13,139
5,Tierra Verde,4.07,27
6,Garnet Valley,4.07,23
7,Santa Barbara,4.05,3829
8,Merchantville,4.04,45
9,Saint Pete Beach,4.03,57


In [ ]:
# Checking constraints
# Reopen connection
conn = sqlite3.connect("data/YelpDB.db")
cursor = conn.cursor()

# ✅ Enable Foreign Key Constraints
cursor.execute("PRAGMA foreign_keys = ON;")  # Ensures constraints are enforced

try:
    cursor.execute("""
        INSERT INTO Review (review_id, user_id, business_id, stars, date)
        VALUES ('review_invalid_1', 'non_existent_user', 'valid_business_id', 5, '2025-01-01');
    """)
    conn.commit()
    print("✅ Inserted successfully (Unexpected!)")
except sqlite3.IntegrityError as e:
    print("❌ Constraint Error Caught:", e)

# Close connection
conn.close()


❌ Constraint Error Caught: UNIQUE constraint failed: Review.review_id


##### 8) Close the Database at End

In [18]:
conn.close()
print("Database connection closed!")

Database connection closed!


##### Submitted by:
###### Amash Rizwan
###### 21F-9116
###### BCS-8D